In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import functools
import sklearn
import math

import pyLDAvis
from pprint import pprint

import bitermplus as btm
import tmplot as tmp
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.decomposition import PCA
%matplotlib inline

#------------
from os import path
from wordcloud import WordCloud, STOPWORDS 
from PIL import Image
from wordcloud import ImageColorGenerator
import re
import pickle
import joblib
import spacy

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk import word_tokenize,sent_tokenize
from nltk.stem.porter import PorterStemmer
import datetime
import warnings

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\Tanisha Batra\anaconda3\lib\site-packages\nltk\decorators.py:69: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  signature = inspect.formatargspec(regargs, varargs, varkwargs, defaults,


In [7]:
raw_df= pd.read_csv(r'G:/Symbi/Dataset/TelecomPOC.csv',encoding='utf-8', parse_dates=[1])
raw_df.head()
neg = raw_df[raw_df.Sentiment=='Negative']

neg = neg.reset_index(drop=True)
text = neg.Tweet.dropna()
print(text)
#raw_df= raw_df[pd.notnull(raw_df['Tweet'])] #Delete rows where Tweet is null.

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        @Vodafone won't not let me port to @JioCare f...
1             @jiocare Jio internet services depressing. 
2       @jiocare #jiofraud, no action in my complaint,...
3       @jiocare Was recharging for Jio 999 plan. When...
4       @jiocare It's been officially 10 days. Yet, no...
                              ...                        
1084    @ViCustomerCare Dear Vi India yesterday i visi...
1085    @ViCustomerCare full day ur compny sms recived...
1086    Dear vodafone your cce told me that I have alr...
1087    @surajit4585 @ViCustomerCare Surprised that yo...
1088    You cannot place a Port Out request on Vodafon...
Name: Tweet, Length: 1088, dtype: object


In [27]:
def clean_tweet(RawSentence):
    """
    removing all hashtags , punctuations, stop_words  and links, also stemming words 
    """
    #RawSentence ="Voice over Wifi is a really good option! Keep up the innovation"    
    txt = RawSentence.lower()
    word_tokens = word_tokenize(txt)    
    #word_tokens = word_tokenize(RawSentence)    
    
    lemma = WordNetLemmatizer()
    nlp = spacy.load('en_core_web_sm') 

    stop_words = set(stopwords.words('english'))
    stop_words.update(["amp", "rt", "cc"]) # adding twitter specific stop words
    #stop_words = stop_words - set(['no', 'not','nil'])  #removing No, not, nil from stop words     
    stop_words.update(["worst","airtelpresence","sunday","monday","tuesday","wednesday","thursday","friday","saturday",
                       "weekend","apparently","extremely","worse","consolation","shouldnt","wont","arent","maybe"
                     "better", "other","instead","look","yesterday","unacceptable","miss","couldnt","depress","let",
                     "thank","actually","frustrate","ridiculous","expect","pathetic","terrible","hathway",
                     "impossible","ask","earlier","literally","years","surprise","horrible","airtel","current","face",
                     "upset","pls","one","two","three","four","empty","please","good","finally","proper","development",
                     "situation","company","believe","question","anymore","consolation","mess","shock","longer","twitter",
                      "completely","finally", "waste","shock","annoy","strand",
                      "important","break","reason", "first","second","third","attempt","officially","absolutely","hello","yet","know","jio",
                      "january","february","march","april","may","june","july","august","september","october","november","december","without",
                      "vodafone","get","vodafoneidea","vicustomercare","vowifi","since","voda","reliancejio","jiocare","time","number"])
   
    txt = re.sub(r"(@\S+)", "", txt)  # remove @
    txt = re.sub(r'\W', ' ', str(txt)) # remove all special characters including apastrophie 
    txt = txt.translate(str.maketrans(' ', ' ', string.punctuation)) # remove punctuations 
    
    txt = re.sub(r'\s+[a-zA-Z]\s+', ' ', txt)   # remove all single characters from within the text (it's -> it s then we need to remove s)
    txt = re.sub(r'\^[a-zA-Z]\s+', ' ', txt) # remove all single characters from beginning of the text 
    txt = re.sub(r'\s+', ' ', txt, flags=re.I) # Substituting multiple spaces with single space
    txt = re.sub(r"(http\S+|http)", "", txt) # remove links 
    
    #txt = re.sub(r'^b\s+', '', txt) # generally used when text is scrapped online. 'b' is prefixed often indicating binary text. This is not reqd for us 
    #txt = [re.sub('\s+', ' ', t) for t  in txt]     # Remove new line characters
    #print(txt)
    #txt = ''.join([i for i in txt if not i.isdigit()]).strip() # remove digits ()
    txt = ' '.join([i for i in txt.split(" ") if i not in stop_words])   # split() is native tokenizer in Python
    #txt = ' '.join(lemma.lemmatize(word) for word in txt.split())           # Used lemmatizer later 
    #txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stop_words ]) # stem & remove stop words
    #soup = BeautifulSoup(txt) #to scrap information from web pages....not needed here
    #txt = soup.get_text
    
    def deEmojify(inputString):
        return inputString.encode('ascii', 'ignore').decode('ascii')
    
    def stem_tokens(tokens, lemmatize):
        lemmatized = []
        for item in tokens:
            lemmatized.append(lemma.lemmatize(item,'v'))
        return lemmatized
    
    txt = deEmojify(txt)
    print(txt)
    tokens =[]
    
    excluded_tags = {"ADJ", "ADV", "ADP", "PROPN","CCONJ","AUX","DET","PRON"}
    included_tags = {"NN", "NNP","NNS","NNPS" ,"JJ","JJR","JJS" }
        
    for token in nlp(txt):
        if token.pos_ not in excluded_tags:
        #if token.pos_ in included_tags:
            tokens.append(token.text)
    
    lemm = stem_tokens(tokens, lemma)
    #tokens = nltk.word_tokenize(txt)
    tokens = [ch for ch in lemm if len(ch)>2] #remove words with character length below 3
    #tokens = [ch for ch in tokens if len(ch)<=15] #remove words with character length above 15 
    
    joined_text = ' '.join(tokens)
        
    return joined_text

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
#raw_df['clean_tweet'] = raw_df.Tweet.apply(clean_tweet)
#print(text)
clean_text=text.apply(clean_tweet)

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        @Vodafone won't not let me port to @JioCare f...
1             @jiocare Jio internet services depressing. 
2       @jiocare #jiofraud, no action in my complaint,...
3       @jiocare Was recharging for Jio 999 plan. When...
4       @jiocare It's been officially 10 days. Yet, no...
                              ...                        
1085    @ViCustomerCare Dear Vi India yesterday i visi...
1086    @ViCustomerCare full day ur compny sms recived...
1087    Dear vodafone your cce told me that I have alr...
1088    @surajit4585 @ViCustomerCare Surprised that yo...
1089    You cannot place a Port Out request on Vodafon...
Name: Tweet, Length: 1089, dtype: object
After removing punctuations    won t not let me port to  for no reason  my second attempt and i don t know what to do  agents are no good  
After removing links   won not let me port to for no reason my second attempt and don know what to do agents are no good 
 won not let me port to for no reason my second attempt and

After removing punctuations  hey  i m facing problem with slow internet connection  i just complained and they replied me that   so many people are using internet on same time   can someone tell me is this the right excuse to give your costumer  
After removing links  hey m facing problem with slow internet connection just complained and they replied me that so many people are using internet on same time can someone tell me is this the right excuse to give your costumer 
hey m facing problem with slow internet connection just complained and they replied me that so many people are using internet on same time can someone tell me is this the right excuse to give your costumer 
hey facing problem slow internet connection complained replied many people using internet someone tell right excuse give costumer 
After removing punctuations  now it s almost 16hours without network don t know what s happening with jio 
After removing links  now it almost 16hours without network don know what happe

After removing punctuations  poor network with 0 00 kb internet speed 
After removing links  poor network with 0 00 kb internet speed 
poor network with 0 00 kb internet speed 
poor network 0 00 kb internet speed 
After removing punctuations  are you even checking the network speeds   please help in this regard
After removing links  are you even checking the network speeds please help in this regard
are you even checking the network speeds please help in this regard
even checking network speeds help regard
After removing punctuations  dear  as i have registered my complaint on 7th april 2021   my complaint no is 313609835442035  but no further action is taken in this matter  and you claim to be a zero complain company  may that is because you give zero important to your complains 
After removing links  dear as have registered my complaint on 7th april 2021 my complaint no is 313609835442035 but no further action is taken in this matter and you claim to be zero complain company may that

After removing punctuations  thanks  we stay in mumbai but ur network is giving feeling of remote area  not available have to always manually search for it 
After removing links  thanks we stay in mumbai but ur network is giving feeling of remote area not available have to always manually search for it 
thanks we stay in mumbai but ur network is giving feeling of remote area not available have to always manually search for it 
thanks stay mumbai ur network giving feeling remote area available always manually search 
After removing punctuations  no network again  why this is happening time and again
After removing links  no network again why this is happening time and again
no network again why this is happening time and again
network happening
After removing punctuations  calls you regarding your complaint and as soon as you pick up the call  then hang up and send a message saying we tried reaching you but couldn t  pathetic excuse for a company 
After removing links  calls you regardi

...

After removing punctuations   day by recharge  costly  but service is so much bad  no network  no help
After removing links   day by recharge costly but service is so much bad no network no help
 day by recharge costly but service is so much bad no network no help
 day recharge costly service much bad network help
After removing punctuations   hi team    my vi number is closed from two days   there are no network  may i know the reason 
After removing links   hi team my vi number is closed from two days there are no network may know the reason 
 hi team my vi number is closed from two days there are no network may know the reason 
 hi team vi closed days network 
After removing punctuations   sir  it is going to be 2 months  only you do this one message  nothing
After removing links   sir it is going to be 2 months only you do this one message nothing
 sir it is going to be 2 months only you do this one message nothing
 sir going 2 months message nothing
After removing punctuations    

After removing punctuations   have been facing potential network issues from many days in my area  can t even talk on call properly  kindly resolve this on utmost priority or will switch the operator now 
After removing links   have been facing potential network issues from many days in my area can even talk on call properly kindly resolve this on utmost priority or will switch the operator now 
 have been facing potential network issues from many days in my area can even talk on call properly kindly resolve this on utmost priority or will switch the operator now 
 facing potential network issues many days area even talk call properly kindly resolve utmost priority switch operator 
After removing punctuations   there is a huge problem of vodafone network please install a tower here people are buying airtel and jio sim due to low networking of vodafone idea please do something village gura  bidhuna  auraiya  uttar pradesh
After removing links   there is huge problem of vodafone network 

After removing punctuations   lets move to jio postpaid       is it possible i have both prepaid and pospaid sim of jio  as  are doing worst job on customer griveances resolution    
After removing links   lets move to jio postpaid is it possible have both prepaid and pospaid sim of jio as are doing worst job on customer griveances resolution 
 lets move to jio postpaid is it possible have both prepaid and pospaid sim of jio as are doing worst job on customer griveances resolution 
 lets move postpaid possible prepaid pospaid sim job customer griveances resolution 
After removing punctuations  i m facing same problem  i m going to port my sim in jio    vi service and internet both are worst    https   t co i2yz8mhuux
After removing links  i facing same problem m going to port my sim in jio vi service and internet both are worst  co i2yz8mhuux
i facing same problem m going to port my sim in jio vi service and internet both are worst  co i2yz8mhuux
facing problem going port sim vi servic

After removing punctuations   i know what update you people will give  same update your team is telling from last 3 months   seriously there is no chnage in the single word  everyone reading the script which  has prepared 
After removing links   know what update you people will give same update your team is telling from last 3 months seriously there is no chnage in the single word everyone reading the script which has prepared 
 know what update you people will give same update your team is telling from last 3 months seriously there is no chnage in the single word everyone reading the script which has prepared 
 update people give update team telling last 3 months seriously chnage single word everyone reading script prepared 
After removing punctuations    the issue never fixed that s what i am communicating to your executive since last 1 month  please keep this complaint open until this issue get fixed
After removing links   the issue never fixed that what am communicating to your exe

After removing punctuations  dear vodafone your cce told me that i have already used my 100  data of daily limit but as per your app it s showing 0 96gb left which one is right and why i m not able to use 4g data indoor     
After removing links  dear vodafone your cce told me that have already used my 100 data of daily limit but as per your app it showing 0 96gb left which one is right and why m not able to use 4g data indoor 
dear vodafone your cce told me that have already used my 100 data of daily limit but as per your app it showing 0 96gb left which one is right and why m not able to use 4g data indoor 
dear cce told already used 100 data daily limit per app showing 0 96gb left right able use 4g data indoor 
After removing punctuations    surprised that your complaint is registered  i have been trying to register my complaint for almost a month now and they refused to do so  they do not even share appllette or nodal officers numbers  switching to jio now
After removing links   su

In [9]:
clean_text.to_csv('clean_tweets.csv',index=False)

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
X, vocabulary, vocab_dict = btm.get_words_freqs(clean_text, max_df=.85, min_df=15,max_features=2000, ngram_range=(1,2))

# Vectorizing documents
docs_vec = btm.get_vectorized_docs(clean_text, vocabulary)

# Generating biterms
Y = X.todense()
biterms = btm.get_biterms(docs_vec, 15)

# INITIALIZING AND RUNNING MODEL
model = btm.BTM(X, vocabulary, T=8, M=10, alpha=500/1000, beta=0.01, win=15, has_background= True)
model.fit(biterms, iterations=500, verbose=True)
p_zd = model.transform(docs_vec,verbose=True)  
print(p_zd) 

# matrix of document-topics; topics vs. documents, topics vs. words probabilities 
matrix_docs_topics = model.matrix_docs_topics_    #Documents vs topics probabilities matrix.
topic_doc_matrix = model.matrix_topics_docs_      #Topics vs documents probabilities matrix.
matrix_topic_words = model.matrix_topics_words_   #Topics vs words probabilities matrix.

# Getting stable topics
print("Array Dimension = ",len(matrix_topic_words.shape))
closest_topics, dist = btm.get_closest_topics(*matrix_topic_words, top_words=100, verbose=True)
stable_topics, stable_kl = btm.get_stable_topics(closest_topics, thres=0.7)

# Stable topics indices list
print(stable_topics)


C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 12%|█▎        | 1/8 [00:00<?, ?it/s]

[[7.79423460e-06 1.08322833e-05 9.09950285e-02 ... 3.14186360e-01
  3.82427430e-01 1.08318643e-05]
 [2.54030326e-06 3.06557250e-05 2.94290594e-09 ... 2.35168864e-05
  5.39140674e-01 6.97389756e-05]
 [6.45091811e-05 2.18496335e-05 1.75799664e-01 ... 3.92825341e-01
  2.59874924e-01 4.50728965e-02]
 ...
 [2.58387746e-05 3.93565915e-01 1.00330837e-06 ... 2.46682855e-01
  2.99633018e-01 3.97313713e-03]
 [6.79360677e-05 5.28891655e-08 3.63169998e-02 ... 2.67459620e-01
  2.64034428e-01 3.68800544e-05]
 [2.76732078e-05 1.33103074e-01 1.19966812e-04 ... 3.19189233e-01
  4.13237882e-01 2.68440303e-08]]
Array Dimension =  2


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [77]:
# INITIALIZING AND RUNNING MODEL
model = btm.BTM(X, vocabulary, T=8, M=10, alpha=50/1000, beta=0.01, win=15)
model.fit(biterms, iterations=500, verbose=True)

p_zd = model.transform(docs_vec,verbose=True)  
#model.transform(self, list docs, unicode infer_type=u'sum_b', bool verbose=True)


C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 1089/1089 [00:00<00:00, 19279.96it/s]


In [78]:
# METRICS
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
coherence = btm.coherence(model.matrix_topics_words_, X, M=10)

print("Perplexity ",perplexity)
print("Coherence ",coherence)
# or
perplexity1 = model.perplexity_
coherence1 = model.coherence_
print("Perplexity1 ",perplexity1)
print("Coherence1 ",coherence1)

Perplexity  95.69940020903734
Coherence  [ -95.38662781 -104.70819151  -93.11622805 -107.05478685  -92.74374166
 -111.02808012 -108.22863831  -76.38544281]
Perplexity1  95.69940020903734
Coherence1  [ -95.38662781 -104.70819151  -93.11622805 -107.05478685  -92.74374166
 -111.02808012 -108.22863831  -76.38544281]


C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [79]:
tmp.report(model=model, docs=clean_text)

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [87]:
#iter_count = model.iterations_
#print("Iterations count ",iter_count )
#Number of iterations the model fitting process has gone through.

Matrix_docs_topics = model.matrix_docs_topics_    #Documents vs topics probabilities matrix.
#print("Matrix_doc_topics ", model.matrix_docs_topics_)

topic_doc_matrix = model.matrix_topics_docs_
#print("Matrix topic docs ",model.matrix_topics_docs_)
#Topics vs documents probabilities matrix.

matrix_topic_words = model.matrix_topics_words_
#print("Topics vs words probabilities Matrix ",model.matrix_topics_words_) 
#Topics vs words probabilities matrix.

#print("Words vs topics probabilities ",model.matrix_words_topics_)
#Words vs topics probabilities matrix.

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
np.random.seed(122334)
#print(np.random.seed(122334))
reference_model = np.random.randint(1, 6)
#print(reference_model)

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [88]:
#print("Number of topics ",model.topics_num_)
#Number of topics.
print("Array Dimension = ",len(matrix_topic_words.shape))
closest_topics, dist = btm.get_closest_topics(*matrix_topic_words, top_words=139, verbose=True)
# Getting stable topics
stable_topics, stable_kl = btm.get_stable_topics(closest_topics, thres=0.7)

# Stable topics indices list
print(stable_topics)

C:\Users\Tanisha Batra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 12%|█▎        | 1/8 [00:00<00:00, 31.92it/s]

Array Dimension =  2


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed